# Clasificación de imágenes usando una red profunda

**Profesor:** Roberto Muñoz <br />
**E-mail:** <rmunoz@metricarts.com> <br />

**Colaborador:** Sebastián Arpón <br />
**E-mail:** <rmunoz@metricarts.com> <br />


Usaremos el dataset de perros y gatos. El dataset completo se puede descargar desde el link https://www.kaggle.com/c/dogs-vs-cats/data

In [ ]:
import tensorflow as tf

tf.enable_eager_execution()

import os
import glob
import requests
import zipfile

import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

Descargamos un dataset pequeño de perros y gatos. Lo descargamos ejecutando el siguiente código

In [ ]:
data_url = "https://metriclearning.blob.core.windows.net/tensorflow/cats_and_dogs_small.zip"

# En caso de querer descargar un dataset un poco mas grande
# use este link https://metriclearning.blob.core.windows.net/tensorflow/cats_and_dogs_medium.zip

data_dir = "data"
data_file = os.path.join(data_dir, "cats_and_dogs_small.zip")

if not os.path.exists(data_dir):
    os.mkdir(data_dir)

response = requests.get(data_url)
response_data = response.content

with open(data_file, 'wb') as f:
    f.write(response_data)

with open(data_file, 'rb') as f:
    zf = zipfile.ZipFile(f)
    zf.extractall(data_dir)

Hacemos un resize de las imágenes a 150 x 150 pixels

In [ ]:
def read_dir(directory):
    cats = glob.glob(os.path.join(directory,"cats") + '/*.jpg')
    dogs = glob.glob(os.path.join(directory,"dogs") + '/*.jpg')
    m_images = cats + dogs
    
    m_labels = []
    m_labels.extend([CAT] * len(cats))
    m_labels.extend([DOG] * len(dogs))
    assert len(m_labels) == len(m_images)
    LABELS_DIMENSIONS = 2
    m_labels = tf.one_hot(m_labels, LABELS_DIMENSIONS)
    print("Encontre %d imagenes y etiquetas en %s" %(len(m_images),directory))
    return m_images, m_labels

def load_image(path_to_image, p_label):
    m_label = p_label
    m_image = tf.read_file(path_to_image)
    m_image = tf.image.decode_jpeg(m_image)
    m_image = tf.image.resize_images(m_image,(150,150))
    m_image = m_image / 255
    return m_image, m_label

In [ ]:
data_dir = "data/cats_and_dogs_small"
train_dir = os.path.join(data_dir, "train")
val_dir = os.path.join(data_dir , "validation")

CAT = 0
DOG = 1

print("Carpeta con imagenes para el entrenamiento: ", train_dir)
print("Carpeta con imagenes para la validación: ", val_dir)
print()

train_images, train_labels = read_dir(train_dir)
val_images, val_labels = read_dir(val_dir)

In [ ]:
i=np.random.randint(len(train_images))

img, label = load_image(train_images[i],"")

print(type(img))
print(type(label))

plt.imshow(img)
print(train_labels[i])

In [ ]:
batch_size = 64
buffer_size = 1000

train_data_set = tf.data.Dataset.from_tensor_slices((train_images,train_labels))
train_data_set = train_data_set.shuffle(buffer_size).map(load_image).batch(batch_size)

val_data_set = tf.data.Dataset.from_tensor_slices((val_images,val_labels))
val_data_set = val_data_set.shuffle(buffer_size).map(load_image).batch(batch_size)

In [ ]:
for i, (x,y) in enumerate(train_data_set):
    print(i)
    print(x.shape)
    print(y.shape)

### Definimos la arquitectura de la red

Usamos el AdamOptimizer como funcion de optimización

In [ ]:
def create_model():

    model = tf.keras.Sequential()

    model.add(tf.layers.Conv2D(16, (5,5), activation=tf.nn.relu, input_shape=(150,150,3)))

    model.add(tf.layers.MaxPooling2D(pool_size=(2,2),strides=2))

    model.add(tf.layers.Conv2D(64,(3,3),activation=tf.nn.relu))
    model.add(tf.layers.MaxPooling2D(pool_size=(2,2),strides=2))
    model.add(tf.layers.Conv2D(128,(3,3),activation=tf.nn.relu))
    model.add(tf.layers.MaxPooling2D(pool_size=(2,2),strides=2))
    model.add(tf.layers.Conv2D(128,(3,3),activation=tf.nn.relu))
    model.add(tf.layers.MaxPooling2D(pool_size=(2,2),strides=2))
    model.add(tf.layers.Flatten())
    model.add(tf.layers.Dense(512,activation=tf.nn.relu))
    model.add(tf.layers.Dense(2,activation=tf.nn.softmax))
    
    model.compile(loss='categorical_crossentropy', 
                  optimizer=tf.train.AdamOptimizer(),
                  metrics=['accuracy'])
    
    return model
    
model = create_model()    
model.summary()

In [ ]:
EPOCHS = 5
batch_loss_list=[]
val_loss_list=[]

for epoch in range(EPOCHS):
    for (batch, (images, labels)) in enumerate(train_data_set):
        batch_loss, batch_accuracy = model.train_on_batch(images.numpy(), labels.numpy())
        batch_loss_list.append(float(batch_loss))
        
        for (dummy, (val_im, val_lab)) in enumerate(val_data_set):
            val_loss, val_accuracy = model.evaluate(val_im.numpy(), val_lab.numpy())
            val_loss_list.append(float(val_loss))
            
        if batch%5 == 0:
            print('Entrenamiento Epoca #%d\t Loss: %.6f\t Accuracy:  %.6f\t'% (epoch+1, batch_loss, batch_accuracy))
            print('Validacion Epoca #%d\t Loss: %.6f\t Accuracy:  %.6f\t' % (epoch+1, val_loss, val_accuracy))

### Graficamos la evolución de la función de costo

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(8,6))

ax.plot(batch_loss_list, label="train")
ax.plot(val_loss_list, label="validation")

plt.xlabel("iteración")
plt.ylabel("Función de costo - train")
plt.legend()

plt.title("Tain dataset")